In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl
output_dir="output"
plots_dir = "plots/finite_cross/"
import os

os.makedirs(plots_dir, exist_ok=True)

from perdiver.distances import *
import perdiver.perdiver as perdiver

In [ ]:
from navground import sim, core

behaviour= "ORCA"
number = 20
side = 16
yaml = f"""
steps: 10000
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Cross
  agent_margin: 0.2
  side: {side}
  target_margin: 1.0
  tolerance: 0.5
  groups:  
    -
      type: thymio
      number: {number}
      control_period: 0.1
      behavior:
        type: {behaviour}
        safety_margin: 0.25 
        horizon: 3
        barrier_angle: 1
      radius: 0.2
      kinematics:
        type: 2WDiff
        max_speed: 0.5
        wheel_axis: 2
      state_estimation:
        type: Bounded
        range: 2.0 
"""
experiment = sim.load_experiment(yaml)
experiment.run()

In [ ]:
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

Record Video

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx
from navground.sim.ui.video import record_video, record_video_from_run

def linear_map(a, b, cmap):
    c = cmx.ScalarMappable(norm=colors.Normalize(vmin=a, vmax=b), cmap=cmap)  
    def f(v):
        r, g, b, _ = c.to_rgba(v)
        return f"#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}"
    return f


fill_map = linear_map(0.0, 1.0, cmap=cmx.RdYlGn)

def f(entity):
    if isinstance(entity, sim.Agent):
        return {'fill': fill_map(entity.behavior.efficacy)}
    return {}
    
record_video(f"{plots_dir}{behaviour}_exp.mp4", run.world, time_step=0.1, duration=60.0, factor=6.0,
             bounds=((-side*0.6, -side*0.6), (side*0.6, side*0.6)), decorate=f, width=500)

In [ ]:
from IPython.display import Video

Video(f"{plots_dir}{behaviour}_exp.mp4", width=200)

In [ ]:
weight = 0.5

In [ ]:
dist_result = perdiver.trajectory_distance_weighted_velocities(ps[:5], twists[:5], weight)

In [ ]:
start_step_list = list(range(0,2400, 500))
fig, ax = plt.subplots(ncols=len(start_step_list), figsize=(6*(len(start_step_list)),6))
xmax, ymax = 0,0
for idx_start, start_step in enumerate(start_step_list):
    shift_time=50
    X, Y = ps[idx_start], ps[idx_start+shift_time]
    vel_X, vel_Y = twists[idx_start], twists[idx_start+shift_time]
    Dist_X = perdiver.distances_weighted_velocities(X, vel_X, weight)
    Dist_Y = perdiver.distances_weighted_velocities(Y, vel_Y, weight)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax[idx_start])
    # plot_matching_diagram_trajectories(ps[start_step:start_step+shift_time], twists[start_step:start_step+shift_time], weight, ax[idx_start], shift_time, color=mpl.colormaps["GnBu"](idx/len(steplist)))
    ax[idx_start].set_title(f"{start_step}")
    xmax = max(xmax, ax[idx_start].get_xlim()[1])
    ymax = max(ymax, ax[idx_start].get_ylim()[1])

for idx_start, start_step in enumerate(start_step_list):
    ax[idx_start].plot([0,max(xmax, ymax)], [0, max(xmax, ymax)], color="gray")
    ax[idx_start].set_xlim(0,xmax)
    ax[idx_start].set_ylim(0,ymax)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
step = 1605
shift_time = 40
X, Y = ps[step], ps[step+shift_time]
vel_X, vel_Y = twists[step], twists[step+shift_time]
Dist_X = perdiver.distances_weighted_velocities(X, vel_X, weight)
Dist_Y = perdiver.distances_weighted_velocities(Y, vel_Y, weight)
Dist_Z = np.minimum(Dist_X, Dist_Y)
perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax)
plt.tight_layout()
plt.savefig(f"per_div_cross_{number}.png")

In [ ]:
start_step_list = list(range(3600,4000, 5))
shift_time=40
fig, ax = plt.subplots(figsize=(8,6))
for idx, step in enumerate(start_step_list):
    X, Y = ps[step], ps[step+shift_time]
    vel_X, vel_Y = twists[step], twists[step+shift_time]
    Dist_X = perdiver.distances_weighted_velocities(X, vel_X, weight)
    Dist_Y = perdiver.distances_weighted_velocities(Y, vel_Y, weight)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    perdiver.plot_matching_diagram(Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(start_step_list)), max_val_diag=8)
    
# Colorbar 
norm = mpl.colors.Normalize(start_step_list[0], vmax=start_step_list[-1])
cmap = mpl.colormaps["GnBu"]
mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
plt.colorbar(mappable=mappable, ax=ax)
plt.tight_layout()
plt.savefig(f"{plots_dir}{behaviour}_{weight}_perdiver.png")

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
X = ps[2000]
ax.scatter(X[:,0], X[:,1], c="black", marker="o", s=20)